In [1]:
try:
    import cupy
    print("CuPy is installed.")
    print("Version:", cupy.__version__)
except ImportError:
    print("CuPy is not installed.")
print("CuPy version:", cupy.__version__)

CuPy is installed.
Version: 13.3.0
CuPy version: 13.3.0


In [2]:
import sys
sys.path.append('..')
from common import config
# GPUで実行する場合は、下記のコメントアウトを消去（要cupy）
# ===============================================
config.GPU = True
# ===============================================
from common.np import *
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from cbow import CBOW
from skip_gram import SkipGram
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb


# ハイパーパラメータの設定
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 2 #デフォルトは10


------------------------------------------------------------
                       GPU Mode (cupy)
------------------------------------------------------------



In [3]:
# ハイパーパラメータの設定
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# データの読み込み
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)

# モデルなどの生成
model = CBOW(vocab_size, hidden_size, window_size, corpus)
# model = SkipGram(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

# 学習開始
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

# 後ほど利用できるように、必要なデータを保存
word_vecs = model.word_vecs
if config.GPU:
    word_vecs = to_cpu(word_vecs)
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'  # or 'skipgram_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)


| epoch 1 |  iter 1 / 9295 | time 0[s] | loss 4.16
| epoch 1 |  iter 21 / 9295 | time 0[s] | loss 4.16
| epoch 1 |  iter 41 / 9295 | time 1[s] | loss 4.15
| epoch 1 |  iter 61 / 9295 | time 1[s] | loss 4.12
| epoch 1 |  iter 81 / 9295 | time 2[s] | loss 4.04
| epoch 1 |  iter 101 / 9295 | time 2[s] | loss 3.92
| epoch 1 |  iter 121 / 9295 | time 2[s] | loss 3.77
| epoch 1 |  iter 141 / 9295 | time 3[s] | loss 3.61
| epoch 1 |  iter 161 / 9295 | time 3[s] | loss 3.48
| epoch 1 |  iter 181 / 9295 | time 4[s] | loss 3.36
| epoch 1 |  iter 201 / 9295 | time 4[s] | loss 3.26
| epoch 1 |  iter 221 / 9295 | time 5[s] | loss 3.16
| epoch 1 |  iter 241 / 9295 | time 5[s] | loss 3.09


KeyboardInterrupt: 